In [2]:
!rm 'OnlineRetail.csv.gz' -f
!wget https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/OnlineRetail.csv.gz

--2018-01-22 03:16:32--  https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/OnlineRetail.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7483128 (7.1M) [application/octet-stream]
Saving to: ‘OnlineRetail.csv.gz’

100%[======================================>] 7,483,128   21.0MB/s   in 0.3s   

2018-01-22 03:16:33 (21.0 MB/s) - ‘OnlineRetail.csv.gz’ saved [7483128/7483128]



In [4]:
loadRetailData = sc.textFile("OnlineRetail.csv.gz")

for row in loadRetailData.take(5):
    print (row)
    
#load data into RDD and view

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850,United Kingdom


In [5]:
#Data shaping and preparing )-;

header = loadRetailData.first()
loadRetailData = loadRetailData.filter(lambda line: line != header).map(lambda l: l.split(","))
#split each line that is not the header on the basis of the comma.. and store it back
for row in loadRetailData.take(5):
    print (row)

['536365', '85123A', 'WHITE HANGING HEART T-LIGHT HOLDER', '6', '12/1/10 8:26', '2.55', '17850', 'United Kingdom']
['536365', '71053', 'WHITE METAL LANTERN', '6', '12/1/10 8:26', '3.39', '17850', 'United Kingdom']
['536365', '84406B', 'CREAM CUPID HEARTS COAT HANGER', '8', '12/1/10 8:26', '2.75', '17850', 'United Kingdom']
['536365', '84029G', 'KNITTED UNION FLAG HOT WATER BOTTLE', '6', '12/1/10 8:26', '3.39', '17850', 'United Kingdom']
['536365', '84029E', 'RED WOOLLY HOTTIE WHITE HEART.', '6', '12/1/10 8:26', '3.39', '17850', 'United Kingdom']


In [8]:
#further cleaning 

#Remove the rows that have incomplete data. 
#Keep only the rows that meet the following criteria:

#The purchase quantity is greater than 0
#The customer ID not equal to 0
#The stock code is not blank after you remove non-numeric characters

import re

loadRetailData = loadRetailData.filter(lambda l: int(l[3]) > 0\
                                and len(re.sub("\D", "", l[1])) != 0 \
                                and len(l[6]) != 0)

print (loadRetailData.take(2))



[['536365', '85123A', 'WHITE HANGING HEART T-LIGHT HOLDER', '6', '12/1/10 8:26', '2.55', '17850', 'United Kingdom'], ['536365', '71053', 'WHITE METAL LANTERN', '6', '12/1/10 8:26', '3.39', '17850', 'United Kingdom']]


In [9]:
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)
# Create a  DF: Sql context...
#Convert each line to a Row.
loadRetailData = loadRetailData.map(lambda l: Row(inv=int(l[0]),\
                                    stockCode=int(re.sub("\D", "", l[1])),\
                                    description=l[2],\
                                    quant=int(l[3]),\
                                    invDate=l[4],\
                                    price=float(l[5]),\
                                    custId=int(l[6]),\
                                    country=l[7]))

# the lambda: for each row.. map to the said column headings

In [11]:
retailDf = sqlContext.createDataFrame(loadRetailData)
print (retailDf.printSchema())
# Create DF and load data


root
 |-- country: string (nullable = true)
 |-- custId: long (nullable = true)
 |-- description: string (nullable = true)
 |-- inv: long (nullable = true)
 |-- invDate: string (nullable = true)
 |-- price: double (nullable = true)
 |-- quant: long (nullable = true)
 |-- stockCode: long (nullable = true)

None


In [12]:
retailDf.registerTempTable("retailPurchases")
sqlContext.sql("SELECT * FROM retailPurchases limit 2").toPandas()
#register the  DF for SQL operations through register temp table

,country,custId,description,inv,invDate,price,quant,stockCode
0,United Kingdom,17850,WHITE HANGING HEART T-LIGHT HOLDER,536365,12/1/10 8:26,2.55,6,85123
1,United Kingdom,17850,WHITE METAL LANTERN,536365,12/1/10 8:26,3.39,6,71053


In [13]:
query = """
SELECT 
    custId, stockCode, 1 as purch
FROM 
    retailPurchases 
group 
    by custId, stockCode"""
retailDf = sqlContext.sql(query)
retailDf.registerTempTable("retailDf")

sqlContext.sql("select * from retailDf limit 10").toPandas()

# removing unneeded columns

,custId,stockCode,purch
0,18074,22224,1
1,13705,21889,1
2,15862,22441,1
3,15862,21592,1
4,12838,22739,1
5,12838,22149,1
6,14078,22548,1
7,14078,22423,1
8,12433,21977,1
9,14696,84360,1


In [15]:
# Splitting the date into training/ crossvalidation/ testing set .8,.1,.1

testDf, cvDf, trainDf = retailDf.randomSplit([.1,.1,.8],1)

print ("trainDf count: ", trainDf.count(), " example: ")
for row in trainDf.take(2): print (row)
print

print ("cvDf count: ", cvDf.count(), " example: ")
for row in cvDf.take(2): print (row)
print

print ("testDf count: ", testDf.count(), " example: ")
for row in testDf.take(2): print (row)
print

trainDf count:  208123  example: 
Row(custId=12359, stockCode=23345, purch=1)
Row(custId=12363, stockCode=20685, purch=1)
cvDf count:  25876  example: 
Row(custId=12349, stockCode=23545, purch=1)
Row(custId=12388, stockCode=22960, purch=1)
testDf count:  26113  example: 
Row(custId=12362, stockCode=22372, purch=1)
Row(custId=12391, stockCode=20985, purch=1)


<function print>

### building recommendation models...
Standard parameters specify data and options. Hyperparameters control the performance of the algorithm.
For ALS algo
Hyperparams are:
The rank represents the number of features. The default value of rank is 10.
The maxIter represents the number of iterations to run the least squares computation. The default value of maxIter is 10.

    Use the training DataFrame to train three models with the ALS algorithm with different values for the rank and maxIter hyperparameters.
    Assign the userCol, itemCol, and ratingCol parameters to the appropriate data columns. 
    Set the implicitPrefs parameter to true so that the algorithm can predict latent factors.

In [17]:

from pyspark.ml.recommendation import ALS

als1 = ALS(rank=3, maxIter=15,userCol="custId",itemCol="stockCode",ratingCol="purch",implicitPrefs=True)
model1 = als1.fit(trainDf)

als2 = ALS(rank=15, maxIter=3,userCol="custId",itemCol="stockCode",ratingCol="purch",implicitPrefs=True)
model2 = als2.fit(trainDf)

als3 = ALS(rank=15, maxIter=15,userCol="custId",itemCol="stockCode",ratingCol="purch",implicitPrefs=True)
model3 = als3.fit(trainDf)

print ("The models are trained")

The models are trained


### Testing the models..

Test the models with the cross validation set and then with the testing data.. 
you know that th emodel is good if the prediction value for the products that the customers already brought is close to 1


In [19]:
#Cleaing the CV DS

from pyspark.sql.functions import UserDefinedFunction
#not sure where the user defined fn is being used.
from pyspark.sql.types import BooleanType
customers = set(trainDf.rdd.map(lambda line: line.custId).collect())
stock = set(trainDf.rdd.map(lambda line: line.stockCode).collect())

print (cvDf.count())
cvDf = cvDf.rdd.filter(lambda line: line.stockCode in stock and line.custId in customers).toDF()
print (cvDf.count())

25876
25846


In [21]:
predictions1 = model1.transform(cvDf)
predictions2 = model2.transform(cvDf)
predictions3 = model3.transform(cvDf)
# the 3 models were set up with different Hyper parameters as defined above...
print( predictions1.take(2))
print (predictions2.take(2))
print (predictions3.take(2))

[Row(custId=16669, stockCode=20735, purch=1, prediction=0.00905850064009428), Row(custId=14606, stockCode=20735, purch=1, prediction=0.03682738170027733)]
[Row(custId=16669, stockCode=20735, purch=1, prediction=0.0033240753691643476), Row(custId=14606, stockCode=20735, purch=1, prediction=0.04877405986189842)]
[Row(custId=14606, stockCode=20735, purch=1, prediction=0.09258811920881271), Row(custId=16669, stockCode=20735, purch=1, prediction=0.007210470270365477)]


In [23]:
meanSquaredError1 = predictions1.rdd.map(lambda line: (line.purch - line.prediction)**2).mean()
meanSquaredError2 = predictions2.rdd.map(lambda line: (line.purch - line.prediction)**2).mean()
meanSquaredError3 = predictions3.rdd.map(lambda line: (line.purch - line.prediction)**2).mean()
    
print ('Mean squared error = %.4f for our first model' % meanSquaredError1)
print ('Mean squared error = %.4f for our second model' % meanSquaredError2)
print ('Mean squared error = %.4f for our third model' % meanSquaredError3)

Mean squared error = 0.7387 for our first model
Mean squared error = 0.6986 for our second model
Mean squared error = 0.6686 for our third model


In [24]:
filteredTestDf = testDf.rdd.filter(lambda line: line.stockCode in stock and line.custId in customers).toDF()
predictions4 = model3.transform(filteredTestDf)
meanSquaredError4 = predictions4.rdd.map(lambda line: (line.purch - line.prediction)**2).mean()
    #
print ('Mean squared error = %.4f for our best model' % meanSquaredError4)

Mean squared error = 0.6693 for our best model


In [25]:
filteredTestDf.take(3)

[Row(custId=12362, stockCode=22372, purch=1),
 Row(custId=12391, stockCode=20985, purch=1),
 Row(custId=12424, stockCode=23034, purch=1)]

In [28]:
filteredTestDf.count()


26085

In [ ]:
# THe MSE of the model.. when run on the filteredTestDf.. is close to the MSE on the model3 => predictions3 result..

## implementing the model model3 
### to predict which product a specific customer might be pruchasing..

In [29]:
from pyspark.sql.functions import lit

stock15544 = set(trainDf.filter(trainDf['custId'] == 15544).rdd.map(lambda line: line.stockCode).collect())
## could have been any customer
# check agains what the map was supposed to do...
userItems = trainDf.select("stockCode").distinct().withColumn('custId', lit(15544)).rdd.filter(lambda line: line.stockCode not in stock15544).toDF()

for row in userItems.take(5):
    print (row.stockCode, row.custId)

21899 15544
22429 15544
22201 15544
22165 15544
21209 15544


In [32]:
#Run the transform function to create a prediction value for each product:

userItems = model3.transform(userItems)

for row in userItems.take(5):
    print (row.stockCode, row.custId, row.prediction)

20735 15544 0.006818763911724091
21220 15544 0.05893325060606003
21700 15544 0.0666210949420929
22097 15544 -0.06284123659133911
22223 15544 0.016460057348012924


In [33]:
# funny how a prediction is negative... :D

In [34]:
# top 5
userItems.registerTempTable("predictions")
query = "select * from predictions order by prediction desc limit 5"

sqlContext.sql(query).toPandas()

,stockCode,custId,prediction
0,21242,15544,0.561971
1,22090,15544,0.549427
2,22326,15544,0.506005
3,21987,15544,0.497307
4,22417,15544,0.472694


In [35]:
stockItems = sqlContext.sql("select distinct stockCode, description from retailPurchases")
stockItems.registerTempTable("stockItems")

query = """
select 
    predictions.*,
    stockItems.description
from
    predictions
inner join stockItems on
    predictions.stockCode = stockItems.stockCode
order by predictions.prediction desc
limit 10
"""
sqlContext.sql(query).toPandas()

,stockCode,custId,prediction,description
0,21242,15544,0.561971,RED RETROSPOT PLATE
1,22090,15544,0.549427,PAPER BUNTING RETROSPOT
2,22326,15544,0.506005,ROUND SNACK BOXES SET OF4 WOODLAND
3,21987,15544,0.497307,PACK OF 6 SKULL PAPER CUPS
4,22417,15544,0.472694,PACK OF 60 SPACEBOY CAKE CASES
5,47590,15544,0.472635,PINK HAPPY BIRTHDAY BUNTING
6,47590,15544,0.472635,BLUE HAPPY BIRTHDAY BUNTING
7,21559,15544,0.470797,STRAWBERRY LUNCH BOX WITH CUTLERY
8,22367,15544,0.468015,CHILDRENS APRON SPACEBOY DESIGN
9,21122,15544,0.464589,SET/10 PINK POLKADOT PARTY CANDLES


In [ ]:
# the resutls match for this sample.. this data set..
# match as in the predicted vs actual.. match